# RECUERDA TENER ANACONDA INSTALADA!
pip install ipykernel

python -m ipykernel install --user --name=BigData

pip install py4j

# INTRODUCCION!
En este capitulo vamos a ver como procesar grandes cantidades de informacion! para esto vamos a usar Apache Spark, un sistema que nos permitira almacenar y procesar grandes cantidades de datos de manera totalmente distributiva

Apache spar es un sistema de computo masivo dise;ado para procesar datos de manera distributiva sobre clusteres de ordenadores. Gracias a su diseno distributivo, Spark puede procesar cantidades de datos del orden de terabytes  o incluso petabytes

Apache Spark esta implementado en el lenguaje de programacion Scala, que combina los paradigmas imperativo y funcional y es ejecutado en la maquina virtual de Java.

Ademas de esto ofrece APIs para Java, Python y R.


El nucleo de Apache Spark esta dividido en:

Spark SQL: Simplifica el manejo de datos masivos, Puede usar sintaxis SQL

Spark Straming: Datos que se deben procesar en tiempo real

Mlib: Alberga todo lo relacionado con el aprendizaje automatico

GraphX: Manejo de Grafos

# Conjuntos de datos distribuidos resilientes (RDDs)

4 caracteristicas de los RDDS

- Estan formados por un conjunto de registros, tambien llamados elementos, todos del mismo tipo. Por ejemplo, si cargamos un fichero de textro se creara un RDD de cadenas de texto, una por cada linea del fichero

- Han sido creados para ser distributivos,

- Son inmutables una vez que se les asigna un valor no son modificables.

- Tienen mucha facilidad a recuperar su estado inicial cuando hay algun problema

## Creación de RDDs

In [1]:
from pyspark import SparkContext, SparkConf
try:
    sc.stop()
except:
    print("✅")
conf = SparkConf().setAppName("MiApp").setMaster("local")
sc = SparkContext(conf=conf)


✅


In [2]:
sc

<SparkContext master=local appName=MiApp>

Lamentablmenete los tipos en python no nos ofrecen mas informacion acerca del tipo de dato de RDD. Unicamente no s dira que es un RDD.

In [3]:
# RDD de 5 enteros
# SRE CREA UN RDD DDE 5 REGISTROS CON LOS NUMEROS DEL 1 AL 5
r = sc.parallelize([1,2,3,4,5])
type(r)

pyspark.rdd.RDD

In [4]:
# RDD de 3 cadenas de texto
# LO MISMO PERO CON STRINGS
r = sc.parallelize(["hola", "hi", "ciao"])
type(r)

pyspark.rdd.RDD

Tambien podemos usar herramientas de python para crear RDD's mas grandes y complejos!

In [5]:
# RDD con los primeros 100 cuadrados perfectos
r = sc.parallelize([i*i for i in range(1,101)])
type(r)

pyspark.rdd.RDD

RECORDEMOS TENER INSTALADO `pip install pyspark` en windows y en nuestro entorno virtual!

In [6]:
r = sc.textFile("../../data/Cap6/file.txt")  # Carga un fichero
print(r.collect(),',\n')
# r = sc.textFile("../../data/Cap6/*.txt")     # Carga todos los ficheros *.txt del directorio
# print(r.collect(),',\n')
# r = sc.textFile("../../data/Cap6/")           # Carga todos los ficheros del directorio
# print(r.collect(),',\n')

['1En el anterior apartado hemos explicado que los RDDs son inmutables', 'y que se crean mediante transformaciones de otros RDDs. Sin embargo,', 'antes de poder comenzar la secuencia de transformaciones necesitamos', 'un RDD (o varios) con los valores iniciales que queremos procesar.', 'Estos RDDs se crearán a partir de valores en memoria del nodo driver,', 'o a partir de ficheros accesibles desde el clúster. En ambos casos se', 'creará un RDD y sus distintas particiones se distribuirán entre los', 'diferentes procesos executor.'] ,



In [7]:
# r = sc.wholeTextFiles("../../data/Cap6/*.txt")
# print(r.collect())

## Acciones
Las acciiones son impacientes, es decrir que al lanzar una accion sobre un RDD esta comenzara inmediatamente su ejecucion de manera distribuida a lo largo del cluster.

### Collect, Take y Count

## Collect
nos sirve para tomar elementos RDD's con poca cantidad de datos! Nos deveulve todos los elementos del RDD

In [8]:
r = sc.parallelize([1,2,3,4,5,6])
print( type(r.collect()) )
r.collect()

<class 'list'>


[1, 2, 3, 4, 5, 6]

## Take
Nos sirve para tomar elementos de RDD's extremadamente grandes, ya que solo nos devuelve los primeros `n` elementos del RDD sin colapsar el proceso driver

In [9]:
r = sc.parallelize(range(1000))
r.take(10)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

En take existen dos metodos que nos ayudan tener en cuenta el orden concreto en que queremos ver los elementos obtenidos

In [10]:
r = sc.parallelize(range(1000))
r.takeOrdered(10, lambda x : -x) # menor a mayor

[999, 998, 997, 996, 995, 994, 993, 992, 991, 990]

In [11]:
r = sc.parallelize(range(1000))
r.takeSample(True, 10) # 10 elementos aleatorios

[906, 331, 845, 994, 21, 309, 953, 461, 743, 432]

## Count
calcula el numero de elementos de un RDD y lo devuelve como entero

In [12]:
r = sc.parallelize(range(1000))
r.count()

1000

# Reduce y Aggregate

la reduccion es una funcion binaria que acepta dos elementos de tipo T y devuelve un valor del mismo tipo T (TxT->T).

Recibe una funcion como argumento, va a seleccionar los dos primeros elementos luego aplica la funcion, y hace esto con los elementos resultantes, hasta que quede un unico elemento!

In [13]:
def suma(x, y):
    return x + y

r = sc.parallelize(range(1,6))
r.reduce(suma)

15

In [14]:
r = sc.parallelize(range(1,6))
r.reduce(lambda x,y: x+y)

15

In [15]:
def multiplica_positivos(x, y):
    if x > 0 and y > 0:
        return x*y
    elif x > 0:
        return x
    elif y > 0:
        return y
    else:
        return 1
    
r = sc.parallelize([-1,2,1,-5,8], 1)
r.reduce(multiplica_positivos)

16

In [16]:
r = sc.parallelize([])
# r.reduce(lambda x,y : x+y) # Esta acción generaría una excepción

In [17]:
r = sc.parallelize([1]) # nos devuleve el mismo elemento
r.reduce(lambda x,y : x+y)

1

Al no estar garantizado el orden, la resta no nos servira para reducir terminos, porque al cambiar las particiones el orden cambia!

In [18]:
r = sc.parallelize(range(3), 1)  # una sola particion
print(r.reduce(lambda x,y: x-y))
r = sc.parallelize(range(3), 2)  # dos particiones
print(r.reduce(lambda x,y: x-y))

-3
1


EN GENERAL LA RESTA Y LA DIVISION NO SE DEBEN USAR PARA LA REDUCCION!

# AGGREGATE
Supongamos que queremos contar todas las letras h que tienen nuestros elementos  de la lista. Para ello, podemos usar una funcion de spark llamada aggregate:

Este aceptara 3 parametros

- Un valor inicial como el acumulador (tipo t)

- Una funcion sqOp para combinar elementos de nuestro RDD (tipo c)

- Una funcion combOp para combinar dos acumuladores (devuelve valor tipo c)

In [19]:
r = sc.parallelize(["hola", "hi", "ciao"])
r.aggregate(0, lambda c, s : c + s.count('h'), lambda c1, c2: c1 + c2)

2

# Salvar RDD's en ficheros

In [20]:
r = sc.parallelize(range(1000), 2)   # 2 particiones
# A veces no funciona!
try:
    r.saveAsTextFile("file:///tmp/file") # Crea dos ficheros part-00000 y part-00001
except:
    print("Ya existe!")
    
# *FALLA si /tmp/file ya existe*

Ya existe!


In [21]:
r = sc.parallelize(range(1000), 1)    # 1 partición
try : # Crea un único fichero part-00000 con todos los números
    r.saveAsTextFile("file:///tmp/unique")
except:
    print("ya existe el archivo")
# *FALLA si /tmp/unique ya existe*

ya existe el archivo


In [22]:
r = sc.parallelize(["hola", "hi", "ciao"])
#r.saveAsTextFile("file:///tmp/file") # lanza una excepción 'Output directory file:/tmp/file already exists'

## Transformaciones

### MAP Y FLATMAP

#### Map 
Nos creara un RDD nuevo, donde cada elemento resultante es el resultado de aplicar la funcion f a un elemento del RDD original

In [23]:
#CON FUNCION ANONIMA
r = sc.parallelize([1,2,3,4])
r2 = r.map(lambda x: x + 1)
r2.collect()

[2, 3, 4, 5]

In [24]:
#FUNCION DEFINIDA
def incrementa(x):
    return x + 1

r = sc.parallelize([1,2,3,4])
r2 = r.map(incrementa)
r2.collect()

[2, 3, 4, 5]

In [25]:
# LEEMOS LAS LONGITUDES DE LOS DATOS Y LOS POENEMOS
#-----------------------------------------------------------
r = sc.parallelize(["hola","hi","ciao"])
r2 = r.map(lambda s: len(s))
r2.collect()

[4, 2, 4]

In [26]:
#LEER ARCHIVOS
import csv
r = sc.parallelize(["1,5,7","8,2,4"])
r2 = r.map(lambda s: list(csv.reader([s]))[0])
#                                      \_->Objeto iterable para el csv reader
r2.collect()

[['1', '5', '7'], ['8', '2', '4']]

In [27]:
# CONVERTIMOS LOS NUMEROS A ENTEROS (ESTABAN COMO STRING)
r3 = r2.map(lambda l: [int(e) for e in l])
r3.collect()

[[1, 5, 7], [8, 2, 4]]

### FLATMAP
A diferencia del map el flatMap aplana los elementos que originalmente son listas!, es decrir, un elemento que originalmente era:

[1,2,3] se convertira a 1,2,3

Hagamos ejemplos similares que con el map!

In [28]:
import csv
r = sc.parallelize(["1,5,7","8,2,4"])
r2 = r.flatMap(lambda s: list(csv.reader([s]))[0])
r2.collect()

#Ahora vemos que nos devuelve una unica lista! porque al aplanar la lista se quita la diferencia entre ellas, y como el FlatMap genera una lista final

['1', '5', '7', '8', '2', '4']

### Filter
El metodo filter recibe como parametro una funcion que calcula la conducion deseada. La cual devolvera Valores Booleanos

In [29]:
def es_primo(x):
    for i in range(2,x):
        if x % i == 0:
            return False
    return True

r = sc.parallelize(range(2,31))
r2 = r.filter(es_primo)
r2.collect()

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]

### RDDs de parejas

In [30]:
r = sc.parallelize([('a',0), ('b', 1),('c',2)])
r2 = r.mapValues(lambda x: x+1)
r2.collect()

[('a', 1), ('b', 2), ('c', 3)]

In [31]:
r = sc.parallelize([('a',0), ('b', 1),('c',2)])
r2 = r.map(lambda p: (p[0], p[1]+1))
r2.collect()

[('a', 1), ('b', 2), ('c', 3)]

In [32]:
import csv
#dividir 
r = sc.parallelize([('a','1,5,7'),('b','8,2')])
r2 = r.flatMapValues(lambda x: list(csv.reader([x]))[0])
r2.collect()

[('a', '1'), ('a', '5'), ('a', '7'), ('b', '8'), ('b', '2')]

### groupByKey
Esto hara que todos los elmetnos con la misma clave se agrupen!

In [33]:
r = sc.parallelize([('a',3.14), ('b', 9.4),('a',2.7)])
r2 = r.groupByKey()
r2.collect()

[('a', <pyspark.resultiterable.ResultIterable at 0x1dbc8467550>),
 ('b', <pyspark.resultiterable.ResultIterable at 0x1dbdf5deeb0>)]

In [34]:
# Para mostrar sus elementos transformamos los iterables en listas
r3 = r2.mapValues(lambda x: list(x))
r3.collect()

[('a', [3.14, 2.7]), ('b', [9.4])]

### reduceByKey
lo usamos para aplicar una funcion de combinacion a todos los valores asiciados a una misma clave

In [35]:
r = sc.parallelize([('a',2), ('b', 1),('a',3)])
r2 = r.reduceByKey(lambda x,y: x+y)
r2.collect()

[('a', 5), ('b', 1)]

## Transformaciones combinando dos RDD's

### Union
Une dos RDD's, si hay datos repetidos, apareceran repetidos

In [36]:
r1 = sc.parallelize([1,2,3,4])
r2 = sc.parallelize([2,4,6])
r3 = r1.union(r2)
r3.collect()

[1, 2, 3, 4, 2, 4, 6]

### distinct
Si Hay elementos repetidos, aparecera unicamente uno de ellos

In [37]:
# Eliminamos duplicados con distinct()
r4 = r3.distinct()
r4.collect()

[2, 4, 6, 1, 3]

### Intersection
Selecciona unicamente los elementos repetidos entre los dos RDD's

In [38]:
r1 = sc.parallelize([1,2,3,4])
r2 = sc.parallelize([2,4,2,6])
r3 = r1.intersection(r2)
r3.collect()

[2, 4]

### Subtract
Selecciona los elementos del primer RDD que no aparecen en el segundo RDD

In [39]:
r1 = sc.parallelize([1,2,3,4,1])
r2 = sc.parallelize([2,6])
r3 = r1.subtract(r2)
r3.collect()

[4, 1, 1, 3]

### Mas tipos de combinaciones!

#### Cartesian
Si el primer RDD tiene N elementos y el segundo M elementos, el resultado sera un RDD con N x M parejas, una por cadda posible combinacion del perimer RDD con el otro RDD

In [40]:
r1 = sc.parallelize([1,2,3])
r2 = sc.parallelize(['a','b'])
#1(a,b), 2(a,b), 3(a,b)
#[(1, 'a'), (1, 'b'), (2, 'a'), (2, 'b'), (3, 'a'), (3, 'b')]
r3 = r1.cartesian(r2)
r3.collect()

[(1, 'a'), (1, 'b'), (2, 'a'), (2, 'b'), (3, 'a'), (3, 'b')]

### Join
Creara una pareja con el valor en el primer RDD seguido del valor en el segundo RDD
Es decir, buscara las claves iguales y las combinara!

In [41]:
r1 = sc.parallelize([('a',1),('b',2),('c',3)])
r2 = sc.parallelize([('b',8),('d',7)])
r3 = r1.join(r2)
r3.collect()

[('b', (2, 8))]

En este caso como hay dos claves iguales, se crean dos elementos con esa misma clave!

In [42]:
# Reunión en presencia de claves duplicadas
r1 = sc.parallelize([('a',1),('b',2),('c',3)])
r2 = sc.parallelize([('b',8),('d',7),('b',0)])
r3 = r1.join(r2)
r3.collect()

[('b', (2, 8)), ('b', (2, 0))]

## Ejemplo de procesamiento de RDD

In [43]:
import csv

############################################################################
## Cargar el fichero CSV original y almacenarlo en un RDD de diccionarios ##
############################################################################

# raw: RDD de listas de cadenas de texto
raw = (
  sc.textFile("../../data/Cap6/titanic.csv") # Leemos el fichero de texto
    .map(lambda s: list(csv.reader([s]))[0]) # Dividimos el CSV en listas
    .filter(lambda l: l[0] != 'PassengerId') # Eliminamos la cabecera
)
print("Instancias totales:", raw.count())

Instancias totales: 891


Funciones para purificar datos

In [44]:
# Función que comprueba si una lista tiene todos los valores en las columnas que nos interesan ([1,2,4,5,6,7,9,11])
def complete(l):
    for i in [1,2,4,5,6,7,9,11]: # Únicamente comprobamos los campos que conservaremos
        if l[i] == '':
            return False
    return True

# Función que transforma la lista de cadenas de texto en diccionarios con enteros, reales o cadenas de texto,
# dependiendo de la columna. Además, descarta las columnas 'PassengerId', 'Name', 'Ticket' y 'Cabin'
def proyect_and_parse(l):
    return {'Survived':int(l[1]), 'Pclass': int(l[2]), 'Sex':l[4], 'Age':float(l[5]), 'SibSp':int(l[6]), 
             'Parch':int(l[7]), 'Fare':float(l[9]), 'Embarked':l[11]
           }

Aplicar funciones definidas recientemente!

In [45]:
# non_null: RDD de diccionarios con valores del tipo adecuado 
non_null = (
  raw.filter(complete)
     .map(proyect_and_parse)
)
print("Instancias completas:", non_null.count(),'\n')

Instancias completas: 712 



### Otras posibles operaciones

In [46]:
###############################################################
## Normalizar las columnas 'Age', 'SibSp', 'Parch' y 'Fare'. ##
###############################################################

# Función que acepta dos diccionarios y combina cada clave usando la función f
# Suponemos que ambos diccionarios tiene las mismas claves
def combine_dicts(f, d1, d2):
    res = dict()
    for k in d1.keys():
        res[k] = f(d1[k], d2[k])
    return res

# Diccionario de valores máximos para cada columna
maximos = non_null.reduce(lambda d1, d2: combine_dicts(max, d1, d2))
# Diccionario de valores mínimos para cada columna
minimos = non_null.reduce(lambda d1, d2: combine_dicts(min, d1, d2))

print("Máximos:", maximos)
print("Mínimos:", minimos)

# Función que procesa todas las claves en 'keys' de un diccionario y las
# normaliza usando los diccionarios de valores máximos 'maxs' y mínimos 'mins'
# El resto de entradas se queda inalterado
def normalize(d, keys, maxs, mins):
    res = dict()
    for (k,v) in d.items():
        if k in keys:
            res[k] = (v - mins[k]) / (maxs[k] - mins[k])
        else:
            res[k] = v
    return res

normalized = non_null.map(lambda d: normalize(d, ['Age','SibSp','Parch','Fare'],  maximos, minimos))
print("Primera entrada:\n",non_null.first())
print("Primera entrada normalizada:\n",normalized.first(),'\n')



############################################################################################
## Transformar columnas 'Sex' y 'Embarked' a valores enteros consecutivos a partir del 0. ##
############################################################################################

# Función que actualiza el conjunto de valores únicos con el valor que contiene el diccionario en
# una columna 'key'
def update_set_dict(s, d, key):
    s.add(d[key]) # Spark permite modificar el primer parámetro y devolverlo
    return s

# Diccionario de conjuntos de valores únicos para la columna 'Sex'
sex_values = normalized.aggregate(set(), 
                            lambda s, d: update_set_dict(s, d, 'Sex'),
                            lambda s1,s2: s1.union(s2)
             )

# Diccionario de conjuntos de valores únicos para la columna 'Embarked'
embarked_values = normalized.aggregate(set(), 
                            lambda s, d: update_set_dict(s, d, 'Embarked'),
                            lambda s1,s2: s1.union(s2)
             )

print("Valores únicos de la columna Sex:", sex_values)
print("Valores únicos de la columna Embarked:", embarked_values)

# Función que acepta un objeto iterable (lista, conjunto, ndarray...) de valores únicos y 
# devuelve un diccionario para traducir valor -> número. Los números naturales están en 
# el rango [0, len(it)) 
def values_to_mapping(it):
    values = sorted(it)
    size = len(values)
    return {k:v for (k,v) in zip(values,range(size))}

# Construye los diccionarios traductores a partir de los conjuntos de valores únicos
sex_mapping = values_to_mapping(sex_values)
embarked_mapping = values_to_mapping(embarked_values)
print("Mapping para la columna Sex:", sex_mapping)
print("Mapping para la columna Embarked:", embarked_mapping,'\n')


# Función que dado un diccionario y una clave, cambia su valor por su traducción según 'mapping'
def dict_translate(d, key, mapping):
    d[key] = mapping[d[key]]
    return d

# Traduce las entradas 'Sex' y 'Embarked' de los diccionarios por sus valores traducidos
titanic = (normalized
             .map(lambda d: dict_translate(d, 'Sex', sex_mapping))
             .map(lambda d: dict_translate(d, 'Embarked', embarked_mapping))
          )
print("Primera entrada preprocesada:\n",titanic.first())

Máximos: {'Survived': 1, 'Pclass': 3, 'Sex': 'male', 'Age': 80.0, 'SibSp': 5, 'Parch': 6, 'Fare': 512.3292, 'Embarked': 'S'}
Mínimos: {'Survived': 0, 'Pclass': 1, 'Sex': 'female', 'Age': 0.42, 'SibSp': 0, 'Parch': 0, 'Fare': 0.0, 'Embarked': 'C'}
Primera entrada:
 {'Survived': 0, 'Pclass': 3, 'Sex': 'male', 'Age': 22.0, 'SibSp': 1, 'Parch': 0, 'Fare': 7.25, 'Embarked': 'S'}
Primera entrada normalizada:
 {'Survived': 0, 'Pclass': 3, 'Sex': 'male', 'Age': 0.2711736617240513, 'SibSp': 0.2, 'Parch': 0.0, 'Fare': 0.014151057562208049, 'Embarked': 'S'} 

Valores únicos de la columna Sex: {'female', 'male'}
Valores únicos de la columna Embarked: {'C', 'Q', 'S'}
Mapping para la columna Sex: {'female': 0, 'male': 1}
Mapping para la columna Embarked: {'C': 0, 'Q': 1, 'S': 2} 

Primera entrada preprocesada:
 {'Survived': 0, 'Pclass': 3, 'Sex': 1, 'Age': 0.2711736617240513, 'SibSp': 0.2, 'Parch': 0.0, 'Fare': 0.014151057562208049, 'Embarked': 2}
